In [1]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
sys.path.append("..")
import numpy as np

CLEAN_PATH = "/home/yifan/dataset/resnet18_with_dropout/pairflip/cifar10/0/"
REF_PATH = "/home/yifan/dataset/resnet18_init/pairflip/cifar10/0"


ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis'

DEVICE='cuda:0'
########## initulize reference data and target data
from alignment.data_preprocess import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH)
tar_datainit = DataInit(CLEAN_PATH,CLEAN_PATH,TAR_EPOCH)
ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()


from alignment.ReferenceGenerator import ReferenceGenerator
gen = ReferenceGenerator(ref_provider=ref_provider, tar_provider=tar_provider,REF_EPOCH=REF_EPOCH,TAR_EPOCH=TAR_EPOCH,ref_model=ref_model,tar_model=tar_model,DEVICE=DEVICE)

absolute_alignment_indicates,predict_label_diff_indicates,predict_confidence_Diff_indicates,high_distance_indicates = gen.subsetClassify(18,0.8,0.3,0.05)


from representationTrans.trans_visualizer_border import visualizer
from singleVis.SingleVisualizationModel import VisModel
from singleVis.projector import TimeVisProjector
model = VisModel(ENCODER_DIMS, DECODER_DIMS)

I = np.eye(512)
projector = TimeVisProjector(vis_model=model, content_path=REF_PATH, vis_model_name=VIS_MODEL_NAME, device="cpu")
vis = visualizer(ref_provider, I,I, np.dot(ref_provider.train_representation(TAR_EPOCH),I), projector, 200,[0,1],'tab10')


/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:01<00:00, 125.63it/s]


NET resnet18_with_dropout
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 437.93it/s]


absolute alignment indicates number: 167 label diff indicates number: 13 confidence diff indicates number: 15 high distance number: 75


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from torch.nn.functional import cosine_similarity


class TransformerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, nhead, num_layers):
        super().__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_dim, nhead, hidden_dim),
            num_layers
        )
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(input_dim, nhead, hidden_dim),
            num_layers
        )

    def forward(self, X, Y):
        X = X.unsqueeze(0)  # Add sequence length dimension
        Y = Y.unsqueeze(0)  # Add sequence length dimension
        encoded_Y = self.encoder(Y)
        decoded_X = self.decoder(X, encoded_Y)
        return encoded_Y.squeeze(0), decoded_X.squeeze(0)  # Remove sequence length dimension



def cka(X, Y):
    X_centered = X - X.mean(axis=0)
    Y_centered = Y - Y.mean(axis=0)
    hsic = np.dot(X_centered, Y_centered.T).mean() ** 2
    var_X = np.dot(X_centered, X_centered.T).mean()
    var_Y = np.dot(Y_centered, Y_centered.T).mean()
    return hsic / (var_X * var_Y)

def F_norm(X, Y):
    m = len(X)
    return ((( X - Y)**2).sum())/m


def knn_overlap(X, Y, k=5):
    nbrs_X = NearestNeighbors(n_neighbors=k).fit(X)
    nbrs_Y = NearestNeighbors(n_neighbors=k).fit(Y)
    X_neighbors = nbrs_X.kneighbors(X, return_distance=False)
    Y_neighbors = nbrs_Y.kneighbors(Y, return_distance=False)
    return np.mean([len(np.intersect1d(x_nbrs, y_nbrs)) for x_nbrs, y_nbrs in zip(X_neighbors, Y_neighbors)])

input_dim = 512
hidden_dim = 1024
nhead = 8
num_layers = 4
learning_rate = 0.001
num_epochs = 50

model = TransformerModel(input_dim, hidden_dim, nhead, num_layers).cuda()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [3]:
def batch_cka_loss(Y, transformed_Y):
    cka_score = cka(Y.detach().numpy(), transformed_Y.detach().numpy())
    cka_loss = 1 - cka_score
    return cka_loss

def batch_knn_loss(Y, transformed_Y, k=10):
    knn_score = knn_overlap(Y.detach().numpy(), transformed_Y.detach().numpy(), k=k)
    knn_loss = 1 - (knn_score + 1)  # Add 1 to the knn_score before calculating the loss
    return knn_loss



In [6]:
X = torch.Tensor(ref_train_data)
Y = torch.Tensor(tar_train_data)
cka_weight = 1.0
knn_weight = 1.0

batch_size = 300
# Assume X and Y are both numpy arrays of size (num_samples, input_dim)
train_data = data_utils.TensorDataset(X, Y)
train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, shuffle=True)
encoder_criterion = nn.MSELoss()
decoder_criterion = nn.MSELoss()
encoder_weight = 0.5
decoder_weight = 0.5
from AlignVis.losses import KNNOverlapLoss, CKALoss, PredictionLoss, ConfidenceLoss

 ##### knn loss
knn_overlap_loss = KNNOverlapLoss(k=10)
##### knn overleap different with target
# knn_loss = knn_overlap_loss(input=y, target=x)

for epoch in range(num_epochs):
    for X_batch, Y_batch in train_loader:
        X_batch = X_batch.cuda()
        Y_batch = Y_batch.cuda()

        optimizer.zero_grad()

        encoded_Y, decoded_X = model(X_batch, Y_batch)
        encoder_loss = F_norm(encoded_Y, X_batch)
        decoder_loss = F_norm(decoded_X, Y_batch)
       
        # Compute CKA and KNN losses on the batch level
        cka_loss = batch_cka_loss(Y_batch.cpu(), encoded_Y.cpu())
        knn_loss = knn_overlap_loss(input=Y_batch.cpu(), target=decoded_X.cpu())

        loss = (
            encoder_weight * encoder_loss
            + decoder_weight * decoder_loss
            # + prediction_weight * prediction_loss
            + cka_weight * cka_loss
            + knn_weight * knn_loss
        )



        loss = encoder_weight * (encoder_loss) + decoder_weight * decoder_loss
        loss.backward()
        optimizer.step()

        # print(f'Epoch: {epoch+1}, Loss: {loss.item()},CKA: {cka_loss.item()}, KNN Overlap: {knn_loss.item()}')

    with torch.no_grad():
        X = X.cuda()
        Y = Y.cuda()
        encoded_Y, decoded_X = model(X, Y)  # Unpack the model output here
        # cka_score = cka(Y.cpu().numpy(), encoded_Y.cpu().numpy())
        knn_score = knn_overlap(Y.cpu().numpy(), decoded_X.cpu().numpy())
     
        print(f'Epoch: {epoch+1}, Loss: {loss.item()}, KNN Overlap: {knn_score}')


/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in double_scalars
/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in double_scalars


NameError: name 'cka_score' is not defined